**Text Classification based on whether the text is positive or negative**

The dataset has been created with two columns Sentence and label(0: Negative sentence, 1: Positive sentence)

##QUESTION 2

In [ ]:
!pip install transformers torch datasets


In [ ]:
import pandas as pd
from datasets import Dataset

# Replace 'your_file.xlsx' with the path to your Excel file
xlsx_file_path = '/content/Text Classification.xlsx'

# Read the Excel file using pandas
df = pd.read_excel(xlsx_file_path)

df.head()

,Sentence,label
0,My family's support means the world to me.,1
1,Exploring new places brings me immense joy.,1
2,Achieving my goals fills me with a sense of ac...,1
3,Receiving unexpected compliments brightens my ...,1
4,Enjoying a delicious meal with loved ones is p...,1


In [ ]:
df

,Sentence,label
0,My family's support means the world to me.,1
1,Exploring new places brings me immense joy.,1
2,Achieving my goals fills me with a sense of ac...,1
3,Receiving unexpected compliments brightens my ...,1
4,Enjoying a delicious meal with loved ones is p...,1
...,...,...
195,The constant struggle of dealing with a slow c...,0
196,Experiencing a sudden and unexpected power out...,0
197,The feeling of being overwhelmed by a never-en...,0
198,Constantly dealing with a perpetually messy wo...,0


In [ ]:
from sklearn.model_selection import train_test_split


train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Check the number of examples in each set
print(f"Number of examples in training set: {len(train_data)}")
print(f"Number of examples in test set: {len(test_data)}")

Number of examples in training set: 160
Number of examples in test set: 40


In [ ]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
model_name = "distilbert-base-uncased"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["Sentence"], truncation=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

In [ ]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy = "epoch",
    logging_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)


In [ ]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.387200,0.697408,0.875000
2,0.411500,0.373488,0.900000
3,0.005900,0.164622,0.975000
4,0.001500,0.263958,0.950000
5,0.000800,0.285999,0.950000


TrainOutput(global_step=100, training_loss=0.16139918986707927, metrics={'train_runtime': 206.2677, 'train_samples_per_second': 3.878, 'train_steps_per_second': 0.485, 'total_flos': 3748413421344.0, 'train_loss': 0.16139918986707927, 'epoch': 5.0})

In [ ]:
# Evaluate the model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

{'eval_loss': 0.2859986424446106, 'eval_accuracy': 0.95, 'eval_runtime': 3.44, 'eval_samples_per_second': 11.628, 'eval_steps_per_second': 1.454, 'epoch': 5.0}


The accuracy is found out to be 95%

Q. Discuss what could be done to improve accuracy.

i. **Hyperparameter Tuning**: We can fine-tune hyperparameters such as learning rate, batch size, and weight decay. A grid search or random search can help find optimal values.

ii. **Data Augmentation**: We can introduce data augmentation techniques during tokenization. For text data, this could involve using synonym replacement, back translation, or other text augmentation strategies to increase diversity in the training set.

iii. **Learning Rate Scheduler**: We can also implement a learning rate scheduler to adjust the learning rate during training. This can help fine-tune the model more effectively.

References

https://huggingface.co/docs/transformers/training

https://chat.openai.com/